## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that.

Fill in your **NAME** and **AEM** below:

In [1]:
NAME = "Ιωάννης Οικονομίδης"
AEM = "XXXX"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [ ]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file.
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not.


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7feb2b0042e0>)

In [ ]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

In [ ]:
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The following code will reduce the number of instances, dealing with the small imbalance of the dataset, as well as reducing the size of the dataset!

In [ ]:
from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, RandomUnderSampler

ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X, y)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res

Resampled dataset shape Counter({0: 1687, 1: 1687})


## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **5-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses two **simple** estimators/classifiers. Test both soft and hard voting and report the results. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   KNN Models  

In [ ]:
### BEGIN SOLUTION
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate

# USE RANDOM STATE!
cls1 = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1) # Classifier #1
cls2 = KNeighborsClassifier(n_jobs=-1) # Classifier #2
soft_vcls = VotingClassifier(estimators=[('lr', cls1), ('knn', cls2),], voting='soft', n_jobs=-1) # Voting Classifier
hard_vcls = VotingClassifier(estimators=[('lr', cls1), ('knn', cls2),], voting='hard', n_jobs=-1) # Voting Classifier

svlcs_scores = cross_validate(soft_vcls, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
s_avg_fmeasure = np.average(svlcs_scores['test_f1_weighted']) # The average f-measure
s_avg_accuracy = np.average(svlcs_scores['test_balanced_accuracy']) # The average accuracy

hvlcs_scores = cross_validate(hard_vcls, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
h_avg_fmeasure = np.average(hvlcs_scores['test_f1_weighted']) # The average f-measure
h_avg_accuracy = np.average(hvlcs_scores['test_balanced_accuracy']) # The average accuracy

### END SOLUTION

print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(n_jobs=-1, random_state=42)),
                             ('knn', KNeighborsClassifier(n_jobs=-1))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.8956 & Balanced Accuracy: 0.8957
Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(n_jobs=-1, random_state=42)),
                             ('knn', KNeighborsClassifier(n_jobs=-1))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8568 & Balanced Accuracy: 0.8583


For both soft/hard voting classifiers the F1 weighted score should be above 0.74 and 0.79, respectively, and for balanced accuracy 0.74 and 0.80. Remember! This should be the average performance of each fold, as measured through cross-validation with 5 folds!

### 1.2 Randomization

You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements. Use the same cross-validation approach as before!

In [ ]:
### BEGIN SOLUTION
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

ens1 = BaggingClassifier(random_state=RANDOM_STATE)
ens2 = RandomForestClassifier(random_state=RANDOM_STATE)
ens3 = AdaBoostClassifier(random_state=RANDOM_STATE)
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

ens1_scores = cross_validate(ens1, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
ens2_scores = cross_validate(ens2, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
ens_3_scores = cross_validate(ens3, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)
tree_scores = cross_validate(tree, X, y, cv=5, scoring=('f1_weighted', 'balanced_accuracy'), n_jobs=-1)

f_measures = dict({'Ensemble with bagging': np.average(ens1_scores['test_f1_weighted']),
                   'Ensemble with random forest': np.average(ens2_scores['test_f1_weighted']),
                   'Ensemble with Ada boosting': np.average(ens_3_scores['test_f1_weighted']),
                   'Simple Decision': np.average(tree_scores['test_f1_weighted'])})
accuracies = dict({'Ensemble with bagging': np.average(ens1_scores['test_balanced_accuracy']),
                   'Ensemble with random forest': np.average(ens2_scores['test_balanced_accuracy']),
                   'Ensemble with Ada boosting': np.average(ens_3_scores['test_balanced_accuracy']),
                   'Simple Decision': np.average(tree_scores['test_balanced_accuracy'])})
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}


### END SOLUTION

print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier: {} -  F1 Weighted: {}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier: {} -  BalancedAccuracy: {}".format(name,round(score,4)))

BaggingClassifier(random_state=42)
RandomForestClassifier(random_state=42)
AdaBoostClassifier(random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier: Ensemble with bagging -  F1 Weighted: 0.8251
Classifier: Ensemble with random forest -  F1 Weighted: 0.8609
Classifier: Ensemble with Ada boosting -  F1 Weighted: 0.8348
Classifier: Simple Decision -  F1 Weighted: 0.7435
Classifier: Ensemble with bagging -  BalancedAccuracy: 0.8251
Classifier: Ensemble with random forest -  BalancedAccuracy: 0.861
Classifier: Ensemble with Ada boosting -  BalancedAccuracy: 0.8349
Classifier: Simple Decision -  BalancedAccuracy: 0.7436


### 1.3 Question

Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

One solution to this problem is to parallelize the gereration of the estimators in a bagging classifier as in such a classifier every estimator is independent from each other and thus can be built in parallel without any problems. Every estimator is generated and trained on its own so as many estimators as the machine can handle can be generated and trained in parallel. This can be achieved in Sklearn by using the n_jobs parameter and setting it to -1 to use all the CPU cores available in the machine.

However, this solution does not work with boosting classifiers as in such classifiers the models are trained sequentially trying to correct the errors of the previous model. This means that every estimator is dependent on the previous one as building the next estimator must wait until the previous one has been generated and trained so that the next one can try to correct its errors. As a result, pparallelization in boosting classifiers is impossible. This is why boosting classifiers in Sklearn have no n_jobs parameter.

## 2.0 Creating the best classifier ##
In the second part of this assignment, we will try to train the best classifier, as well as to evaluate it using stratified cross valdiation.

### 2.1 Good Performing Ensemble

In this part of the assignment you are asked to train a good performing ensemble, that is able to be used in a production environment! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy, using 10-fold stratified cross validation, of your final classifier. Can you achieve a balanced accuracy over 88%, while keeping the training time low? (Tip 1: You can even use a model from the previous parts, but you are advised to test additional configurations, and ensemble architectures, Tip 2: If you try a lot of models/ensembles/configurations or even grid searches, in your answer leave only the classifier you selected as the best!)

In [ ]:
### BEGIN SOLUTION
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

# The following is the code of the grid searches used to find the best hyperparameters of the three estimators used by the voting classifier

# rf_param_grid = {'n_estimators': [500], 'max_depth': [5, 10, 15, 20, 25], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': [2, 3, 5, 7, 10], 'random_state': [RANDOM_STATE]}
# rf_grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_param_grid, scoring='f1_weighted', n_jobs=-1)
# rf_grid_search.fit(X, y)
# print(f'Random forest params: {rf_grid_search.best_params_}')

# lr_param_grid = {'C': [0.001, 0.005, 0.01, 0.1, 0.5], 'max_iter': [1000], 'random_state': [RANDOM_STATE]}
# lr_grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=lr_param_grid, scoring='f1_weighted', n_jobs=-1)
# lr_grid_search.fit(X, y)
# print(f'Logistic regression params: {lr_grid_search.best_params_}')

# knn_param_grid = {'n_neighbors': [2, 3, 5, 7, 10]}
# knn_grid_search = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=knn_param_grid, scoring='f1_weighted', n_jobs=-1)
# knn_grid_search.fit(X, y)
# print(f'K-nearest neighbors params: {knn_grid_search.best_params_}')

rf_cls = RandomForestClassifier(n_estimators=500, max_depth=20, criterion='entropy', max_features='sqrt', min_samples_leaf=2, random_state=RANDOM_STATE, n_jobs=-1)
lr_cls = LogisticRegression(C=0.005, max_iter=1000, random_state=RANDOM_STATE)
knn_cls = KNeighborsClassifier(n_neighbors=3)

best_cls = VotingClassifier(estimators=[('rf', rf_cls), ('lr', lr_cls), ('knn', knn_cls)], voting='soft', n_jobs=-1)

weighted_f1_scores = []
balanced_accuracies = []
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
for train_index, test_index in skf.split(X, y):
    X_train_fold, y_train_fold, X_test_fold, y_test_fold = X.loc[train_index, :], y[train_index], X.loc[test_index, :], y[test_index]
    best_cls.fit(X_train_fold, y_train_fold)
    y_pred = best_cls.predict(X_test_fold)
    weighted_f1_scores.append(f1_score(y_test_fold, y_pred, average='weighted'))
    balanced_accuracies.append(balanced_accuracy_score(y_test_fold, y_pred))

best_fmeasure = sum(weighted_f1_scores) / len(weighted_f1_scores)
best_accuracy = sum(balanced_accuracies) / len(balanced_accuracies)
### END SOLUTION

print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score: 0.9098645015693585 & Balanced Accuracy: 0.9099024373062834


LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

The final classifier is a soft voting classifier which uses the three following estimators:

1. Random forest with 500 estimators, 20 max depth, the entropy criterion, sqrt max features and 2 minimum samples per leaf.
2. Logistic regression with C=0.005 and 1000 max iterations
3. K-nearest neighbors with k=3 nearest neighbors

The process I followed is the following:

As a start, I knew from trying different classifiers in exercise 1.1 that Logistic Regression was the best performing classifier in this dataset between Logistic Regression, KNN and simple Decision Trees, with KNN being the second best classifier between those two. I also knew from exercise 1.2 that random forest was the best tree ensemble classifier in this dataset between bagging, random forest and Adaboost.

Additionally, you said we should keep the training time low and I know a random forest with a relatively low number of 500 estimators, logistic regression with a reasonable max number of iterations and KNN without a huge k value do not have a particularly long training time compared to other classifiers, so they are perfect candidates to use in an ensemble classifier whose training must be short.

As such, I decided to start with a single random forest without tuned hyperparameters which didn't give good enough metrics. Next I created a hard voting classifier between the previous random forest and a new logistic regression classifier which was also not tuned. This also didnt give good enough metrics, thus I tested the same voting classifier but with soft voting.

This voting classifier gave good enough metrics but I though I could still improve it relatively easily. To do this, I also added a K-nearest neighbors classifier to the previous soft voting classifier with k=5. This slightly improved the metrics but I wanted to improve the voting classifier's performance further by tuning the hyperparameters of the three estimators using grid search.

To achieve this, I run grid search on the random forest which takes a long time (more than an hour in Google collab) because of using 500 estimators which improves performance over using 100 estimators. I also run grid search on both the logistic regression and the KNN clasifier. I have left the grid search code in comments at the start of this exercise so that you can see the exact hyperparameter values I tested.

I used the default grid search cross validation method so it performs 5-fold stratified cross validation to find the best hyperparameters. If you want you can run the grid search yourself to check it. Then I used the best hyperparameters returned by the three grid searches for the respective estimators of the voting classifier. The best hyperparameters are the ones of the estimators in the final voting classifier which were also mentioned at the start of this answer.

As you can see, the weighted F1 score of the final soft voting classifier is 0.9098645015693585 and its balanced accuracy is 0.9099024373062834. The training time of the final voting classifier in Google collab is about 5 minutes without including the grid search. It is worth mentioning that reducing the number of trees in the random forest estimator of the final voting classifier to a lower number like 100 lowers the voting classifier's training time significantly at the cost of slightly lower metrics. So if 5 minutes is too long a training time, change the number of estimators to 100 and consider that my final best classifier.

I want to mention that even though I have used the random_state=RANDOM_STATE parameter as you mentioned, if I close and reopen the notebook in Google collab, the resulting metric values change even when using the random_state parameters. However, they do stay the same if the block is run multiple times without closing the notebook. So if you run the exercise 2.1 code block yourself, it might give slighlty better or worse metrics than the ones I have reported here. I don't know how to fix that. I also wrote this in the project's feedback portion.

### 2.2 Question
 What other ensemble architectures you tried, and why you did not choose them as your final classifier?

YOUR ANSWER HERE

As I already explained which architectures I tested in the previous exercise 2.1 and why I changed them, I will report the results of each architecture I tested before I landed in the final one here:

1. Random forest classifier: macro f1 = 0.8577, balanced accuracy = 0.8577, short train time.
2. Hard voting between random forest and logistic regression: macro f1 = 0.8741, balanced accuracy = 0.8746, short train time.
3. Soft voting between random forest and logistic regression: macro f1 = 0.8914, balanced accuracy = 0.8915, short train time.
4. Soft voting between random forest, logistic regression and knn (k=5): macro f1 = 0.9009, balanced accuracy = 0.9010, short train time.
5. Soft voting between random forest, logistic regression and knn with tuned hyperparameters (this is the final classifier and its hyperparameters were presented in exercise 2.1): macro f1 = 0.9098, balanced accuracy = 0.9099, medium train time.

### 2.3 Setup the Final Classifier
Finally, in this last cell, set the cls variable to either the best model as occured by the stratified cross_validation, or choose to retrain your classifier in the whole dataset (X, y). There is no correct answer, but try to explain your choice. Then, save your model using pickle and upload it with your submission to e-learning!

In [ ]:
import pickle

### BEGIN SOLUTION
# I'm using the previous best soft voting classifier that was trained during the 10-fold stratified cross_validation in exercise 2.1 as it already achieves good enough metrics
# (above 88% balanced accuracy and weighted F1 score). I'm not retraining it on the whole dataset (X, y) here, so if you want to create the pickle file yourself, then the exercise
# 2.1's block of code needs to be run first.
cls = best_cls

# save with pickle
file_name = "best_classifier.pickle"
pickle.dump(cls, open(file_name, 'wb'))
### END SOLUTION


# load
cls = pickle.load(open(file_name, "rb"))

test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

# We are going to run the following code
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both metrics should aim above 82%! This is going to be tested by us! Make sure your cross validation or your retrained model achieves high balanced accuracy and f1_score (based on 2.1) (more than 88%) as it should achieve at least 82% in our unknown test set!


Please provide your feedback regarding this project! Did you enjoy it?

In [ ]:
# YOUR ANSWER HERE

# I had one problem. Even though I used the random_state parameter with the
# RANDOM_STATE value where possible, I still got different results when running
# the exact same code after closing the notebook and reopening it. However,
# while it was open in Google collab, the results were the same every time I run
# the code, but if I closed and reopened the notebook file, the results were
# different from the previous ones but same when re-running the code as long as
# I didn't close the notebook.